In [1]:
import csv
import cv2
import numpy as np
import h5py
import json
import numpy as np
import sklearn 

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers import MaxPooling2D
from keras.optimizers import Adam

relu = 'relu'

samples = []
with open('own_training_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
            
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
from sklearn.utils import shuffle
def generator(samples, batch_size=4):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            measurements = []
            for line in samples:
                center_path = line[0]
                left_path = line[1]
                right_path = line[2]
#Adding center images
                filename_c = center_path.split('/')[-1]
                current_path_c = './own_training_data/IMG/' + filename_c
                image_c = cv2.imread(current_path_c)
                images.append(image_c)
#Adding left images
                filename_l = left_path.split('/')[-1]
                current_path_l = './own_training_data/IMG/' + filename_l
                image_l = cv2.imread(current_path_l)
                images.append(image_l)
#Adding right images
                filename_r = right_path.split('/')[-1]
                current_path_r = './own_training_data/IMG/' + filename_r
                image_r = cv2.imread(current_path_r)
                images.append(image_r)
#Adding steering angles from all of the three views
                correction=0.2
                steer = line[3]
                steer_left=float(steer)+float(correction)
                steer_right=float(steer)-float(correction)
                measurements.append(steer)
                measurements.append(steer_left)
                measurements.append(steer_right)
            augmented_images, augmented_measurements = [], []
            for image, measurement in zip(images, measurements):
                augmented_images.append(image)
                augmented_measurements.append(measurement)
                augmented_images.append(cv2.flip(image,1))
                augmented_measurements.append(float(measurement)*-1.0)

            X_train = np.array(augmented_images)
            Y_train = np.array(augmented_measurements)
            yield shuffle(X_train, Y_train)
            
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=4)
validation_generator = generator(validation_samples, batch_size=4)

#Build model
model = Sequential()
#Preprocess incoming data
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(1,1))))

#Concolutional layers based on NVIDIA network
model.add(Convolution2D(6, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(relu))
model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
#model.add(Convolution2D(36, 5, 5, border_mode='same', subsample=(2, 2)))
#model.add(Activation(relu))
#model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
#model.add(Convolution2D(48, 5, 5, border_mode='same', subsample=(2, 2)))
#model.add(Activation(relu))
#model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
#model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(2, 2)))
#model.add(Activation(relu))
#model.add(MaxPooling2D(pool_size=(1, 1), strides=(1, 1)))
model.add(Flatten())
#model.add(Dense(100))
#model.add(Activation(relu))
#model.add(Dense(50))
#model.add(Activation(relu))
#model.add(Dense(10))
#model.add(Activation(relu))
model.add(Dense(1))
model.summary()
model.compile(optimizer=Adam(0.0001), loss="mse")

model.fit_generator(train_generator, samples_per_epoch= len(samples), 
                    validation_data=validation_generator, 
                    nb_val_samples=len(samples)*0.2, nb_epoch=8)
model.save('model.h5', True)

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 65, 318, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 33, 159, 6)    456         cropping2d_1[0][0]               
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 33, 159, 6)    0           convolution2d_1[0][0]            
___________________________________________________________________________________________

ResourceExhaustedError: OOM when allocating tensor with shape[7314,160,320,3]
	 [[Node: truediv = Div[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_lambda_input_1_0, truediv/y)]]

Caused by op 'truediv', defined at:
  File "/home/sandorgoda/anaconda3/envs/carnd-term1-cpu/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sandorgoda/anaconda3/envs/carnd-term1-cpu/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sandorgoda/anaconda3/envs/carnd-term1-cpu/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sandorgoda/anaconda3/envs/carnd-term1-cpu/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sandorgoda/anaconda3/envs/carnd-term1-cpu/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  Fil